# Heart Disease Risk Prediction: Logistic Regression

## Step 1: Load and Prepare the Dataset

**Goal**: Load the Heart Disease dataset, clean it, and prepare it for training a logistic regression model.

**What we'll do**:
1. Load CSV into pandas DataFrame
2. Binarize target (Presence → 1, Absence → 0)
3. Exploratory Data Analysis (EDA)
4. Handle missing values/outliers
5. Select features
6. Split data (70/30 stratified)
7. Normalize features

In [14]:
# Standard imports for data science work
import numpy as np      # Numerical operations, arrays
import pandas as pd     # Data manipulation, DataFrames
import matplotlib.pyplot as plt  # Visualization

# Configure plots to look cleaner
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

## 1.1 Load the Dataset

**Why pandas?** It's the standard tool for tabular data. A DataFrame is like an Excel spreadsheet in Python - rows are samples, columns are features.


In [ ]:
# Load the CSV file
# pd.read_csv() reads a CVS file and returns a DataFrame
df = pd.read_csv("dataset/Heart_Disease_Prediction.csv")

#Quick look at the data
print(f"Dataset shape: {df.shape}") # (rows, columns)
print(f"Samples: {df.shape[0]}, Features: {df.shape[1]}")
df.head() # Show first 5 rows